In [183]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [184]:
df = pd.read_csv("train.csv")
new_cols = ['C' + str(i) for i in range(1, 14 + 1)] + ['X' + str(i) for i in range(1, 19 + 2)]
new_cols[-1] = 'Y'
df.columns = new_cols[:num_cols]
# -1 = not happening (negative), 1 = happening (positive)
#df['Y'].replace({-1 : 0}, inplace=True, regex=True)

df.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,Y
0,V1,V1,V1,V1,V1,V1,V2,V1,V3,V5,...,0.352,0.073,-0.092,1.098,0.034,1.160,0.401,0.037,0.216,1
1,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-1.933,-0.536,-0.092,1.098,0.034,0.716,0.401,0.724,0.216,-1
2,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-0.762,-0.536,-0.092,1.098,0.034,0.716,0.401,0.712,0.216,-1
3,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.505,-0.536,-0.092,0.182,0.034,0.716,0.401,0.393,0.216,1
4,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,0.409,-0.536,-0.092,0.731,0.034,0.716,0.401,0.724,0.216,1


In [185]:
features = df.loc[:, df.columns != 'Y']
numerical_features = df.select_dtypes(include=['float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns
target = df['Y']

In [192]:
from sklearn.impute import SimpleImputer
numerical_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
numerical_imputer.fit(df[numerical_features])
df[numerical_features] = numerical_imputer.transform(df[numerical_features])

categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_imputer.fit(df[categorical_features])
df[categorical_features] = categorical_imputer.transform(df[categorical_features])

encoded_df = pd.get_dummies(df)

In [187]:
encoded_df.to_csv("export.csv", index=False)

In [194]:
X_train, X_test, y_train, y_test = train_test_split(encoded_df, labels, test_size=0.3)
X_train

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,C12_V1,C12_V2,C12_V3,C13_V1,C13_V2,C13_V3,C13_V4,C13_V5,C14_V1,C14_V2
326,-0.557,1.371,-1.092000,-1.299,1.611,-1.805000,0.120000,-0.026,1.450,-0.007,...,0,0,1,1,0,0,0,0,0,1
319,-0.004,1.371,-0.617000,-0.282,-0.328,0.133000,2.093000,-0.844,-0.984,0.568,...,0,1,0,1,0,0,0,0,1,0
208,1.102,-0.490,-1.092000,-1.064,-0.975,-0.015171,0.019816,-0.844,-0.680,1.234,...,0,1,0,1,0,0,0,0,0,1
74,-0.004,-0.738,0.007324,-0.516,-0.458,-0.513000,-0.099000,1.347,-0.272,-1.014,...,1,0,0,1,0,0,0,0,1,0
238,-1.663,0.379,0.823000,0.736,0.189,0.456000,-0.099000,-0.844,0.842,0.784,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,-0.004,-1.111,0.823000,-1.064,-0.458,1.425000,-1.196000,-0.844,-0.071,0.586,...,0,0,1,1,0,0,0,0,1,0
183,-0.004,0.627,-0.617000,0.501,-1.880,0.133000,0.019816,0.777,-1.440,-0.025,...,0,1,0,1,0,0,0,0,1,0
163,-1.110,0.254,-0.127000,-0.516,0.189,1.425000,0.019816,-0.041,0.994,-0.007,...,1,0,0,0,1,0,0,0,0,1
26,-0.557,-0.986,-0.127000,0.501,2.258,1.425000,-0.319000,-0.844,-0.984,0.568,...,1,0,0,1,0,0,0,0,1,0
